In [ ]:
!pip install datasets openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pya

## 1. 데이터 전처리

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from openai import OpenAI

In [ ]:
# Load the dataset from Hugging Face
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [ ]:
subset = dataset.select(range(5000)).to_pandas()[['id', 'sql_prompt', 'sql_context', 'sql']]

In [ ]:
subset.head()

,id,sql_prompt,sql_context,sql
0,5097,What is the total volume of timber sold by eac...,"CREATE TABLE salesperson (salesperson_id INT, ...","SELECT salesperson_id, name, SUM(volume) as to..."
1,5098,List all the unique equipment types and their ...,CREATE TABLE equipment_maintenance (equipment_...,"SELECT equipment_type, SUM(maintenance_frequen..."
2,5099,How many marine species are found in the South...,"CREATE TABLE marine_species (name VARCHAR(50),...",SELECT COUNT(*) FROM marine_species WHERE loca...
3,5100,What is the total trade value and average pric...,"CREATE TABLE trade_history (id INT, trader_id ...","SELECT trader_id, stock, SUM(price * quantity)..."
4,5101,Find the energy efficiency upgrades with the h...,"CREATE TABLE upgrades (id INT, cost FLOAT, typ...","SELECT type, cost FROM (SELECT type, cost, ROW..."


In [ ]:
client = OpenAI(api_key="여러분의 Key값")

In [ ]:
system = '''## 지시사항
1. 이 데이터는 text-to-sql 데이터입니다.
2. 테이블 명세와 실제 sql문을 참고하여 영어 text를 한글 text로 번역하세요.

시작!'''

In [ ]:
user_prompt = []

for context, prompt, sql in \
  zip(subset['sql_context'].to_list(), subset['sql_prompt'].to_list(), subset['sql'].to_list()):
  user_prompt.append('테이블: ' + context + '\nSQL 쿼리: ' +  sql + '\n영어 텍스트: ' + prompt + '\n한글 텍스트:')

print(user_prompt[0])

테이블: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');
SQL 쿼리: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;
영어 텍스트: What is the total volume of timber sold by each salesperson, sorted by salesperson?
한글 텍스트:


In [ ]:
result_lst = []

for user in tqdm(user_prompt[:3]):
  response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
      {"role": "system", "content": system},
      {"role": "user", "content": user}
    ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

100%|██████████| 3/3 [00:11<00:00,  3.83s/it]


In [ ]:
result_lst

['각 영업사원이 판매한 목재의 총량을 영업사원별로 정렬하여 보여주세요.',
 '장비 유형별로 고유한 장비 유형과 해당 장비 유형의 총 유지 보수 빈도를 equipment_maintenance 테이블에서 나열하십시오.',
 '남해양에서 발견되는 해양 생물 종은 몇 종인가요?']

```python
result_lst = []

for user in tqdm(user_prompt):
  response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[
      {"role": "system", "content": system},
      {"role": "user", "content": user}
    ],
    temperature=0
  )
  result_lst.append(response.choices[0].message.content)

subset['korean_text'] = result_lst
subset.to_csv('text-to-sql.csv', index=False, encoding='utf-8-sig')
```

위 과정을 통해 5000개의 데이터를 전부 번역해서 `text-to-sql.csv` 파일로 저장했다고 가정해봅시다.

## 2. LLM 학습 포맷으로 전처리

In [ ]:
df = pd.read_csv('/content/text-to-sql.csv')

In [ ]:
df

,id,sql_prompt,sql_context,sql,korean_text
0,5097,What is the total volume of timber sold by eac...,"CREATE TABLE salesperson (salesperson_id INT, ...","SELECT salesperson_id, name, SUM(volume) as to...","각 영업사원이 판매한 목재의 총량은 무엇이며, 영업사원별로 정렬된 목록은 무엇인가요?"
1,5098,List all the unique equipment types and their ...,CREATE TABLE equipment_maintenance (equipment_...,"SELECT equipment_type, SUM(maintenance_frequen...",equipment_maintenance 테이블에서 모든 고유 장비 유형과 해당하는 ...
2,5099,How many marine species are found in the South...,"CREATE TABLE marine_species (name VARCHAR(50),...",SELECT COUNT(*) FROM marine_species WHERE loca...,남해양에서 발견되는 해양 생물 종은 몇 종인가요?
3,5100,What is the total trade value and average pric...,"CREATE TABLE trade_history (id INT, trader_id ...","SELECT trader_id, stock, SUM(price * quantity)...",trade_history 테이블에서 각 트레이더와 주식별 총 거래 가치와 평균 가격...
4,5101,Find the energy efficiency upgrades with the h...,"CREATE TABLE upgrades (id INT, cost FLOAT, typ...","SELECT type, cost FROM (SELECT type, cost, ROW...",가장 높은 비용을 가진 에너지 효율 개선 업그레이드와 그 종류를 찾으세요.
...,...,...,...,...,...
4995,29213,What is the total number of beauty products so...,CREATE TABLE product_sales (country VARCHAR(25...,"SELECT country, SUM(product_count) as total_pr...",각 국가에서 판매된 뷰티 제품의 총 수량은 얼마입니까?
4996,29214,What is the average time to fill positions for...,"CREATE TABLE Applications (ApplicationID int, ...","SELECT e.Department, p.JobLevel, AVG(DATEDIFF(...",각 부서별로 직급에 따라 포지션 채용까지 평균적으로 얼마나 걸리는지 알려주세요.
4997,29215,How many public awareness campaigns were launc...,"CREATE TABLE campaigns (campaign_id INT, name ...","SELECT country, COUNT(campaign_id) AS campaign...",각 국가별로 시작된 공익 캠페인은 몇 개인가요?
4998,29216,What was the total sales revenue for each drug...,"CREATE TABLE sales (drug VARCHAR(20), region V...","SELECT drug, SUM(CASE WHEN region = 'North' TH...",2023년 2분기 각 약품의 총 매출액을 지역별로 구분하여 알려주세요.


LLM은 입력과 출력의 형태로 학습 데이터가 구성된다. LLM에 입력으로 넣을 프롬프트와 LLM이 생성하기를 원하는 출력을 구성해보자.

In [ ]:
instruction_lst = []
output_lst = []

for context, sql, korean_text in \
  zip(df['sql_context'].to_list(), df['sql'].to_list(), df['korean_text'].to_list()):
  instruction_lst.append('입력 텍스트: ' + korean_text + '\n\nDDL statements:\n' +  context + '\n\n위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.')
  output_lst.append('쿼리 작성: ' + sql)

In [ ]:
print(instruction_lst[0])

입력 텍스트: 각 영업사원이 판매한 목재의 총량은 무엇이며, 영업사원별로 정렬된 목록은 무엇인가요?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.


In [ ]:
print(output_lst[0])

쿼리 작성: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;


In [ ]:
df['instruction'] = instruction_lst
df['output'] = output_lst
df['input'] = '' # 라마팩토리 형식을 맞춰주기 위해서 임의로 추가한 열

In [ ]:
# DataFrame에서 'instruction'과 'output' 열을 딕셔너리 리스트로 변환
data_to_save = df[['instruction', 'input', 'output']].to_dict(orient='records')

# JSON 파일로 저장
with open('text_to_sql_data.json', 'w', encoding='utf-8') as f:
    json.dump(data_to_save, f, ensure_ascii=False, indent=4)